In [ ]:
import pandas as pd
import numpy as np

In [ ]:
train = pd.read_csv("../input/dacon/train.csv")
test = pd.read_csv("../input/dacon/test.csv")
display(train,test)

In [ ]:
all_data = pd.concat([train,test])
all_data

In [ ]:
# income(연봉) /  DAYS_EMPLOYED -> 어떤 사람이 회사에서 돈을 많이 버는가?, 회사에서 주는 연봉 + 이 사람이 들어간지 얼마안됐는데 값이 크다면 연봉상승률이 높다. 값이 작으면 연봉상승률이 작다.
all_data["pay"] = all_data["income_total"] / all_data["DAYS_EMPLOYED"]
all_data

In [ ]:
# income(연봉) / family_size -> 부양될 가족이 너무 많다면 안좋다. 연체비용을 갚기가 어렵다.
all_data["family_pay"] = all_data["income_total"] / all_data["family_size"]
all_data

In [ ]:
#income(연봉) *  DAYS_EMPLOYED -> 이 사람이 지금까지 얼마나 벌었는가?
all_data["year_pay"] = all_data['income_total'] * all_data["DAYS_EMPLOYED"]

In [ ]:
all_data["occyp_type"] = all_data["occyp_type"].fillna("null")
all_data["edu_job"] = all_data["edu_type"] + "_" + all_data["occyp_type"]

In [ ]:
all_data["edu_income"] = all_data["edu_type"] + "_" + all_data["income_type"]

In [ ]:
all_data["family_house"] = all_data["family_type"] + "_" + all_data["house_type"]

In [ ]:
pd.options.display.max_columns = 999
edu_income = all_data.groupby("edu_type")["income_total"].agg(["count","mean","median","std","min","max","sum"])
all_data = all_data.join(edu_income,on="edu_type")
all_data

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
plt.figure(figsize=(25,8))
sns.boxplot(all_data["family_type"],all_data["income_total"],showfliers = False)

In [ ]:
all_data["null_marry"] = np.where(((all_data["occyp_type"] == "null") & (all_data["family_type"] == "Married")),1,0)

In [ ]:
all_data["worst_income"] = np.where(((all_data["income_type"] == "Working") & (all_data["income_total"] < 186685)),1,0)

In [ ]:
all_data["pay"].mean()

In [ ]:
all_data["edu_type"] = all_data["edu_type"].replace({"Academic degree":"null"})

In [ ]:
pd.options.display.max_rows = 999
all_data.corr()["credit"].sort_values(ascending=False)

In [ ]:
# 재산
all_data["has_job"] = all_data["DAYS_EMPLOYED"] != 365243
all_data["money"] = all_data["car"].replace({"Y":1,"N":0}) + all_data["reality"].replace({"Y":1,"N":0}) * 2 + all_data["phone"] + all_data["has_job"] 

In [ ]:
all_data["family_type"].value_counts()

In [ ]:
all_data = all_data.drop(columns=["has_job"])

In [ ]:
all_data["income_diff"] = all_data["income_total"] - all_data["mean"] * 1.5

In [ ]:
# 연봉 -> 표준편차
all_data["income_std"] = np.sqrt(((all_data["income_total"] - all_data["income_total"].mean())**2) / len(all_data["income_total"]))

In [ ]:
# 나이가 많은 데 .. 직장에도 오래 있었다면?
all_data["DAYS_plus"] = all_data["DAYS_BIRTH"] + all_data["DAYS_EMPLOYED"] 

In [ ]:
all_data["DAYS_Square"] = np.sqrt(all_data["DAYS_BIRTH"] ** 2 + all_data["DAYS_EMPLOYED"] ** 2)

In [ ]:
all_data["DAYS_Square_minus"] = np.sqrt(all_data["DAYS_BIRTH"] ** 2 - all_data["DAYS_EMPLOYED"] ** 2)

In [ ]:
# 취업을 얼마나 빨리 했는지 
all_data["NoEmployed"] = all_data["DAYS_BIRTH"] - all_data["DAYS_EMPLOYED"] * 2

In [ ]:
all_data["gender_car_reality"] = all_data["gender"] + "_" + all_data["car"] + "_" + all_data["reality"]

In [ ]:
all_data["job_email"] = all_data["occyp_type"] + "_" + all_data["email"].astype("str")

In [ ]:
all_data2 = all_data.drop(columns=["index","credit","FLAG_MOBIL"])
all_data2

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
for i in all_data2.columns[all_data2.dtypes == "object"]:
    all_data2[i] = le.fit_transform(all_data2[i])
all_data2

In [ ]:
train2 = all_data2[:len(train)]
test2 = all_data2[len(train):]
display(train2,test2)

In [ ]:
# 하이퍼 파라미터 튜닝
# 그리드 서치 (경우의 수가 많음, 시간 소요 문제)
# 랜덤 서치 (그리드서치 보단 낫지만 모든 경우의 수 찾는 것을 막음, 시간 소요 문제)
# 베이지안 튜닝(최신)
# 파이썬 으로 모델링
# import xgboost as xgb
# params = {"tree_method":"gpu_hist","learning_rate" : 0.02, "num_class" : 3, "objective" : "multi:softprob", "eval_metric" : "mlogloss","reg_alpha" : 1e-05}
# xgb_train = xgb.DMatrix(train2,train["credit"]) # 데이터 선언
# result_xgb = xgb.cv(params,xgb_train, num_boost_round = 50, early_stopping_rounds = 50, nfold = 5, stratified = True)

In [ ]:
#result_xgb["test-mlogloss-mean"].min() # 최적의 점수

In [ ]:
# def xgb_bayesian(max_depth,colsample_bytree,subsample,colsample_bylevel,learning_rate):
#     params = {"tree_method":"gpu_hist","learning_rate" : learning_rate, "num_class" : 3, 
#               "objective" : "multi:softprob", "eval_metric" : "mlogloss",
#               "max_depth": int(max_depth),"colsample_bytree" : colsample_bytree,
#               "subsample" : subsample, "colsample_bylevel" : colsample_bylevel,
#               }   
#     xgb_train = xgb.DMatrix(train2,train["credit"]) # 데이터 선언
#     result_xgb = xgb.cv(params,xgb_train, num_boost_round = 50000, 
#                         early_stopping_rounds = 50, nfold = 4, stratified = True)
#     return -result_xgb["test-mlogloss-mean"].min()

In [ ]:
# from bayes_opt import BayesianOptimization
# bo = BayesianOptimization(xgb_bayesian, {"max_depth":(10,20),"colsample_bytree":(0.2,0.7),"subsample":(0.4,0.8),
#                                          "colsample_bylevel":(0.3,1),"learning_rate":(0.02,0.05)})
# bo.maximize(init_points = 20, n_iter = 500)
# colsample_level 이 먼저

In [ ]:
n = 0
result = 0 
from sklearn.model_selection import StratifiedKFold
skf = StratifiedKFold(n_splits = 20,random_state = 82, shuffle = True) 
from xgboost import XGBClassifier
xgb = XGBClassifier(tree_method = "gpu_hist",n_estimators= 50000,learning_rate = 0.02,max_depth = 19,subsample=0.7,colsample_bytree=0.8,
                    colsample_bylevel=0.6,colsample_bynode = 0.3,reg_alpha=1e-05,reg_lambda = 1.5)
best_score = 0
for train_index, valid_index in skf.split(train2,train["credit"]):
    X_train , X_valid = train2.iloc[train_index] , train2.iloc[valid_index]
    y_train , y_valid = train["credit"].iloc[train_index] , train["credit"].iloc[valid_index]    
    xgb.fit(X_train,y_train,eval_set=[(X_valid,y_valid)],early_stopping_rounds = 50,verbose=50) 
    n += 1
    print(f"{n}회")
    best_score += xgb.best_score/20
    result += xgb.predict_proba(test2)/20


In [ ]:
best_score 

In [ ]:
sub = pd.read_csv("../input/dacon/sample_submission.csv")
sub

In [ ]:
sub.iloc[:,1:] = result
sub

In [ ]:
sub.to_csv("sub_최종.csv",index=False)